In [60]:
from faker import Faker
import pandas as pd
import random
import os
import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO, filename="add_1000_orders_logging.log",filemode="w")

payments= pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\payment.csv')
orders= pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\orders.csv')

merged_df=payments.merge(orders,on='payment_id',how='left',indicator=True)

payment_date_converted=pd.DatetimeIndex(merged_df['payment_date']) #convert from object to datetime payment_date_converte

merged_df['Year']=payment_date_converted.year
#merged_df['Quarter']=
merged_df['Month']=payment_date_converted.month  

merged_df.sort_values(by='payment_date',ascending=True,inplace=True)
merged_df.drop_duplicates(subset='payment_id',keep='first',inplace=True)#removing duplicates with same payment_id
columns_todrop=['customer_id_x','emp_id','created_at','payment_type', #removing columns that are not needed
                'order_id','item_id','customer_id_y','delivery',
                'delivery_address_id','item_quantity',
                'customer_modifications','delivery_notes','_merge']
merged_df.drop(columns=columns_todrop,axis=1,inplace=True)

columns=['cafe_id','payment_date','Year','Month','payment_id','amount']
merged_df=merged_df[columns]
merged_df['payment_date']=pd.to_datetime(merged_df['payment_date'])
merged_df['Year']=merged_df['Year'].astype('str')
merged_df['Month']=merged_df['Month'].astype('str')
#using .loc to change all values in column 'Month' to 2 digits
merged_df['Month'] = merged_df['Month'].str.zfill(2)
merged_df['Year_Month']=merged_df['Year']+'-'+merged_df['Month']

#rearranging columns and ordering by cafe_id and Year_Month
first_column = merged_df.pop('Year_Month') 
merged_df.insert(0, 'Year_Month', first_column)
first_column = merged_df.pop('cafe_id') 
merged_df.insert(0, 'cafe_id', first_column)
merged_df.sort_values(by=['cafe_id','Year_Month'],ascending=True,inplace=True)

#calculating total monthly revenue,monthly average check,monthly number of orders in each location 
cafe_locations_monthly_df=merged_df.groupby(['cafe_id','Year_Month','Year','Month']).agg(
    
    what_the_avg_check_was_this_month=('amount','mean'),
    number_of_orders_this_month=('payment_id','count'),
    monthly_revenue_total=('amount','sum')
).reset_index()
cafe_locations_monthly_df['what_the_avg_check_was_this_month']=pd.DataFrame(cafe_locations_monthly_df['what_the_avg_check_was_this_month'].apply(lambda x:round(x,2)))

#calculating  average revenue for each month of the year (Jan-Dec) in each location 
cafe_locations_monthly_avg_df=cafe_locations_monthly_df.groupby(['cafe_id','Month']).agg(
   avg_revenue_month_of_the_year_by_location=('monthly_revenue_total','mean')
).reset_index()
cafe_locations_monthly_avg_df['avg_revenue_month_of_the_year_by_location']=pd.DataFrame(cafe_locations_monthly_avg_df['avg_revenue_month_of_the_year_by_location'].apply(lambda x:round(x,2)))


#calculating  average monthly revenue in each location 
cafe_locations_avg_df=cafe_locations_monthly_df.groupby(['cafe_id']).agg(
   avg_monthly_revenue_by_location=('monthly_revenue_total','mean')
).reset_index()
cafe_locations_avg_df['avg_monthly_revenue_by_location']=pd.DataFrame(cafe_locations_avg_df['avg_monthly_revenue_by_location'].apply(lambda x:round(x,2)))

#joining all 3 dataframes 
cafe_locations_monthly_revenue_df=pd.merge(cafe_locations_monthly_df,cafe_locations_monthly_avg_df,how='left',on=['cafe_id','Month'])
cafe_locations_monthly_revenue_df.reset_index()
cafe_locations_df=pd.merge(cafe_locations_monthly_revenue_df,cafe_locations_avg_df,how='left',on=['cafe_id'])
cafe_locations_df.reset_index(drop=True,inplace=True)

#adding running total by month for each location (starts over in january of each year)
cafe_locations_df['Running_total_by_month']=pd.DataFrame(cafe_locations_df.groupby(['cafe_id','Year'])['monthly_revenue_total'].cumsum(
).reset_index(drop=True))



#adding moving monthly averages by location (average is calculated for last 3 months)
Moving_average=cafe_locations_df['monthly_revenue_total'].rolling(window=3).mean().apply(lambda x: round(x,2)).reset_index(drop=True)
cafe_locations_df['Moving_average_last_3_month']=Moving_average
cafe_locations_df.head()
cafe_locations_df.loc[:,['cafe_id','Year_Month','Year','Month','what_the_avg_check_was_this_month',
                         'number_of_orders_this_month','monthly_revenue_total',
                         'Running_total_by_month','Moving_average_last_3_month',
                         'avg_revenue_month_of_the_year_by_location','avg_monthly_revenue_by_location']]


#downloading the dataframe as .csv file
cafe_locations_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by location\made_using_Python_pandas\Monthly_revenues.csv',index=False)

#exporting annual revenue for each cafe location in a separate .csv
Annual_revenue_df=pd.DataFrame(cafe_locations_df.groupby(['cafe_id','Year']).apply(lambda x: x['monthly_revenue_total'].sum())).reset_index()
Annual_revenue_df.reset_index(drop=True,inplace=True)
Annual_revenue_df.rename(columns={0:'Annual revenue'},inplace=True)

#adding percentage of the total annual revenue by location 
Annual_revenue_df=pd.DataFrame(Annual_revenue_df.loc[:,['Year','cafe_id','Annual revenue']])
Total_annual_revenue_df=pd.DataFrame(Annual_revenue_df.groupby(['Year']).agg(
    Total=('Annual revenue','sum')
)).reset_index()

Annual_revenue_df=pd.merge(Annual_revenue_df,Total_annual_revenue_df,how='left',on='Year')
Annual_revenue_df['Percentage_from_annual_total']=(Annual_revenue_df['Annual revenue']/Annual_revenue_df['Total']*100).apply(lambda x:round(x,2))
Annual_revenue_df.drop(['Total'],axis=1,inplace=True)

Annual_revenue_df.sort_values(by='Year',inplace=True)
Annual_revenue_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by location\made_using_Python_pandas\Annual_revenues.csv',index=False)



C:\Users\SOFYA\AppData\Local\Temp\ipykernel_1864\3234502455.py:91: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  Annual_revenue_df=pd.DataFrame(cafe_locations_df.groupby(['cafe_id','Year']).apply(lambda x: x['monthly_revenue_total'].sum())).reset_index()


In [62]:
#Creating  files with monthly revenues separately for each of the locations:
cafe_1_df=cafe_locations_df[cafe_locations_df['cafe_id']==1]
cafe_1_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by location\made_using_Python_pandas\Cafe_1_monthly_revenues.csv',index=False)
cafe_2_df=cafe_locations_df[cafe_locations_df['cafe_id']==2]
cafe_2_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by location\made_using_Python_pandas\Cafe_2_monthly_revenues.csv',index=False)
cafe_3_df=cafe_locations_df[cafe_locations_df['cafe_id']==3]
cafe_3_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by location\made_using_Python_pandas\Cafe_3_monthly_revenues.csv',index=False)
cafe_4_df=cafe_locations_df[cafe_locations_df['cafe_id']==4]
cafe_4_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by location\made_using_Python_pandas\Cafe_4_monthly_revenues.csv',index=False)
cafe_5_df=cafe_locations_df[cafe_locations_df['cafe_id']==5]
cafe_5_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by location\made_using_Python_pandas\Cafe_5_monthly_revenues.csv',index=False)

,cafe_id,Year_Month,Year,Month,what_the_avg_check_was_this_month,number_of_orders_this_month,monthly_revenue_total,Running_total_by_month,Moving_average_last_3_month,avg_revenue_month_of_the_year_by_location,avg_monthly_revenue_by_location
0,1,2020-02,2020,02,55.92,1,55.92,55.92,NaN,1087.07,1320.43
1,1,2020-03,2020,03,34.15,37,1263.48,1319.40,NaN,1166.80,1320.43
2,1,2020-04,2020,04,37.21,31,1153.64,2473.04,824.346667,1343.83,1320.43
3,1,2020-05,2020,05,28.22,36,1015.82,3488.86,1144.313333,1469.89,1320.43
4,1,2020-06,2020,06,32.57,33,1074.74,4563.60,1081.400000,1362.42,1320.43
...,...,...,...,...,...,...,...,...,...,...,...
301,5,2024-11,2024,11,35.30,44,1553.16,15922.44,1468.960000,1485.66,1325.80
302,5,2024-12,2024,12,34.60,36,1245.56,17168.00,1496.933333,1404.79,1325.80
303,5,2025-01,2025,01,33.22,39,1295.53,1295.53,1364.750000,1427.53,1325.80
304,5,2025-02,2025,02,42.27,28,1183.60,2479.13,1241.563333,1215.98,1325.80
